In [1]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
%cd "/content/gdrive/My Drive/Colab Notebooks/MRNet/"

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/MRNet


In [0]:
import os
import numpy as np
import glob
import csv
import keras
import scipy
from partition import partition
from data_generator import MRNet_data_generator
from keras.layers import Conv2D,Input, MaxPool2D, AveragePooling2D, Dropout, Dense, Flatten
from keras.preprocessing.image import ImageDataGenerator
from PIL import Image
from matplotlib.pyplot import imshow
import random
from vgg_model_generator import MRNet_vgg_model
from VGG_TL_model_generator import MRNet_vgg_tl_model, validation_Callback
from LR_model import LR_model
from LR_generator import MRNet_LR_data_generator
import tensorflow as tf
from numpy.random import seed
from sklearn.utils import class_weight
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.metrics import auc as roc_auc

# seed(1)
# from tensorflow import set_random_seed
# tf.random.set_seed(2)

Using TensorFlow backend.


In [0]:
# training of a single model
label_type = 'meniscus'
exam_view = 'axial'
model, cbs = MRNet_vgg_tl_model(1, 0.000005,combination=[label_type, exam_view])
# print(model(tf.ones((12, 24, 224, 224, 3))).shape)

model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_vgg_TL/"+label_type+"/"+exam_view+"/weights.03.hdf5")
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['train'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['train'][exam_view])
neg = total - pos
class_count_train = {0: neg, 1: pos}
class_weight_train = {0: total/neg, 1: total/pos}
pos = 0
for e in p['test'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['test'][exam_view])
neg = total - pos
class_count_test = {0: neg, 1: pos}
class_weight_test = {0: total/neg, 1: total/pos}

# print(class_count_train, class_count_test)
batch_size = 1
train_data_gen = MRNet_data_generator(data_path, p, l,class_count=None , batch_size=1,model="vgg",data_type='train', label_type=label_type, exam_type=exam_view, scale_to=(224,224), aug_size=0, keep_original=0)
test_data_gen = MRNet_data_generator(data_path, p, l, class_count=None, batch_size=1,model="vgg",data_type='test', label_type=label_type, exam_type=exam_view, scale_to=(224,224), aug_size=0)
vcb = validation_Callback(model, test_data_gen)
cbs.append(vcb)
model.fit(x=train_data_gen, epochs=20, verbose=True, callbacks=cbs,shuffle=False, class_weight=class_weight_train)

In [0]:
# evaluation
label_type = 'meniscus'
exam_view = 'axial'
model, cbs = MRNet_vgg_tl_model(1, 0.00001,combination=[label_type, exam_view])

data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['valid'][exam_view]:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['valid'][exam_view])
neg = total - pos
class_count_valid = {0: neg, 1: pos}
batch_size = 1

combinations = [
                ["abnormal", "axial"],
                ["abnormal", "coronal"],
                ["abnormal", "sagittal"],
                ["ACL", "axial"],
                ["ACL", "coronal"],
                ["ACL", "sagittal"],
                ["meniscus", "axial"],
                ["meniscus", "coronal"],
                ["meniscus", "sagittal"],
]

for c in combinations:
  label_type = c[0]
  exam_view = c[1]
  model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_vgg_TL/"+label_type+"/"+exam_view+"/top.hdf5")
  valid_data_gen = MRNet_data_generator(data_path, p, l,class_count=None , shuffle=1, batch_size=1,model="vgg",data_type='valid', label_type=label_type, exam_type=exam_view, scale_to=(224,224), aug_size=0)
  tp = 0
  tn = 0
  fp = 0
  fn = 0
  y_true = []
  y_score = []
  for i in range(len(valid_data_gen)):
    x, y = next(valid_data_gen)
    _y = model.predict_classes(x, batch_size=1, verbose=0)
    _y_score = model.predict_proba(x, batch_size=1)
    y_score.append(_y_score[0][0])
    y_true.append(y[0])
    if _y[0][0] == 1:
      if _y[0][0] == y[0]:
        tp += 1
      else:
        fp += 1
    else:
      if _y[0][0] == y[0]:
        tn += 1
      else:
        fn += 1
  precision = tp / (tp +fp)
  recall = tp / (tp + fn)
  f1 = 2*((precision*recall)/(precision+recall))
  y_score = np.array(y_score)
  y_true = np.array(y_true)
  fpr, tpr, _ = roc_curve(y_true, y_score)
  auc = roc_auc(fpr, tpr)
  print ("combination: ", c, "tp = ", tp, " fp = ", fp, " tn = ", tn, " fn = ", fn, " accuracy = ", (tp+tn)/(len(valid_data_gen)), " F1 = ", f1, " auc = ", auc)

Initializing Data Generator:
factor :  1
model:  vgg
data type:  valid
Combination:  abnormal  and  axial
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/valid
Number of inputs:  120
input size:  (224, 224)
combination:  ['abnormal', 'axial'] tp =  81  fp =  4  tn =  21  fn =  14  accuracy =  0.85  F1 =  0.9  auc =  0.8770526315789474
Initializing Data Generator:
factor :  1
model:  vgg
data type:  valid
Combination:  abnormal  and  coronal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/valid
Number of inputs:  120
input size:  (224, 224)
combination:  ['abnormal', 'coronal'] tp =  90  fp =  10  tn =  15  fn =  5  accuracy =  0.875  F1 =  0.9230769230769231  auc =  0.9069473684210526
Initializing Data Generator:
factor :  1
model:  vgg
data type:  valid
Combination:  abnormal  and  sagittal
data path:  /content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0/valid
Number of inputs:  120
input size:  (224, 224)
combin

In [0]:
label_type = 'abnormal'
model_axial, _ =  MRNet_vgg_tl_model(1, 0.00001,combination=[label_type, "axial"])
model_coronal, _ =  MRNet_vgg_tl_model(1, 0.00001,combination=[label_type, "coronal"])
model_sagittal, _ =  MRNet_vgg_tl_model(1, 0.00001,combination=[label_type, "sagittal"])
model_axial.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_vgg_TL/"+label_type+"/axial/top.hdf5")
model_coronal.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_vgg_TL/"+label_type+"/coronal/top.hdf5")
model_sagittal.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_vgg_TL/"+label_type+"/sagittal/top.hdf5")
models = [model_axial, model_coronal, model_sagittal]

data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)

pos = 0
for e in p['train']['axial']:
  if l[e][label_type] == 1:
    pos += 1
total = len(p['train']['axial'])
neg = total - pos
class_count_train = {0: neg, 1: pos}
class_weight_train = {0: total/neg, 1: total/pos}
# 0001 => {axial: 90%, coronal: 40%, sagittal: 60%} => 80% abnormal
data_gen_train = MRNet_LR_data_generator(data_path, p, l, models, scale_to=(224, 224), label_type=label_type)
test_data_gen = MRNet_LR_data_generator(data_path, p, l, models, scale_to=(224, 224), data_type="test", label_type=label_type)


Initializing LR Data Generator:
loading data..
1017/1017 [==============================] - 2649s 3s/step
Initializing LR Data Generator:
loading data..
113/113 [==============================] - 293s 3s/step


In [0]:
print(class_weight_train)

{0: 5.242268041237113, 1: 1.2357229647630619}


In [0]:
lr_model, cbs = LR_model(0.00001, label=label_type)
lr_model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_vgg_TL/"+label_type+"/top_3.hdf5")

vcb = validation_Callback(lr_model, test_data_gen)
cbs.append(vcb)
lr_model.fit(x=data_gen_train, epochs=15, verbose=True, callbacks=cbs,shuffle=False, class_weight=class_weight_train)

Epoch 1/15
1017/1017 [==============================] - 4s 4ms/step - loss: 0.4981 - tp: 408.1091 - fp: 0.5457 - tn: 97.0816 - fn: 3.2635 - accuracy: 0.9941 - precision: 0.9990 - recall: 0.9936 - auc: 0.9950

Epoch 00001: saving model to /content/gdrive/My Drive/Colab Notebooks/MRNet/training_vgg_TL/abnormal/weights.01.hdf5

 validation: tp =  85  fp =  10  tn =  13  fn =  5  accuracy =  0.8672566371681416 F1 Score =  0.918918918918919 

Epoch 2/15
1017/1017 [==============================] - 4s 4ms/step - loss: 0.4969 - tp: 1225.8073 - fp: 1.4503 - tn: 288.6913 - fn: 10.0511 - accuracy: 0.9924 - precision: 0.9988 - recall: 0.9918 - auc: 0.9998

Epoch 00002: saving model to /content/gdrive/My Drive/Colab Notebooks/MRNet/training_vgg_TL/abnormal/weights.02.hdf5

 validation: tp =  85  fp =  10  tn =  13  fn =  5  accuracy =  0.8672566371681416 F1 Score =  0.918918918918919 

Epoch 3/15
1017/1017 [==============================] - 4s 4ms/step - loss: 0.4958 - tp: 2037.6362 - fp: 2.6873 -

In [0]:
label_type = 'meniscus'
model_axial, _ =  MRNet_vgg_tl_model(1, 0.00001,combination=[label_type, "axial"])
model_coronal, _ =  MRNet_vgg_tl_model(1, 0.00001,combination=[label_type, "coronal"])
model_sagittal, _ =  MRNet_vgg_tl_model(1, 0.00001,combination=[label_type, "sagittal"])
model_axial.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_vgg_TL/"+label_type+"/axial/top.hdf5")
model_coronal.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_vgg_TL/"+label_type+"/coronal/top.hdf5")
model_sagittal.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_vgg_TL/"+label_type+"/sagittal/top.hdf5")
models = [model_axial, model_coronal, model_sagittal]



data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"
p, l = partition(data_path)
data_gen_valid = MRNet_LR_data_generator(data_path, p, l, models, scale_to=(224, 224), data_type="valid",label_type=label_type)






# data_gen_train = MRNet_LR_data_generator(data_path, p, l, models, scale_to=(224, 224))

# lr_model.fit(x=data_gen_train,validation_data=data_gen_valid, epochs=5, verbose=True, callbacks=cbs,shuffle=False, class_weight=class_weight_train)


Initializing LR Data Generator:
loading data..
120/120 [==============================] - 89s 743ms/step


In [0]:
lr_model, cbs = LR_model(0.001)
lr_model.load_weights("/content/gdrive/My Drive/Colab Notebooks/MRNet/training_vgg_TL/"+label_type+"/top.hdf5")

tp = 0
tn = 0
fp = 0
fn = 0
y_true = []
y_score = []
for i in range(len(data_gen_valid)):
  x, y = next(data_gen_valid)
  _y = lr_model.predict_classes(x, batch_size=1, verbose=0)
  _y_score = lr_model.predict_proba(x, batch_size=1)
  y_score.append(_y_score[0][0])
  y_true.append(y[0])
  if _y[0][0] == 1:
    if _y[0][0] == y[0]:
      tp += 1
    else:
      fp += 1
  else:
    if _y[0][0] == y[0]:
      tn += 1
    else:
      fn += 1
precision = tp / (tp +fp)
recall = tp / (tp + fn)
f1 = 2*((precision*recall)/(precision+recall))
y_score = np.array(y_score)
y_true = np.array(y_true)
fpr, tpr, _ = roc_curve(y_true, y_score)
auc = roc_auc(fpr, tpr)
print ("label: ", label_type, "tp = ", tp, " fp = ", fp, " tn = ", tn, " fn = ", fn, " accuracy = ", (tp+tn)/(len(valid_data_gen)), " F1 = ", f1, " auc = ", auc)

label:  meniscus tp =  44  fp =  26  tn =  42  fn =  8  accuracy =  0.7166666666666667  F1 =  0.721311475409836  auc =  0.8000565610859728


In [0]:
data_path = "/content/gdrive/My Drive/Colab Notebooks/MRNet/dataset/MRNet-v1.0"

with open(data_path+"/validation_exams.txt", "rb") as fp:
  v_set = pickle.load(fp)
os.mkdir(data_path+'/test/')
os.mkdir(data_path+'/test/'+'axial/')
os.mkdir(data_path+'/test/'+'coronal/')
os.mkdir(data_path+'/test/'+'sagittal/')

for e in v_set:
  os.rename(data_path + '/train/'+'axial/' + e + '.npy', data_path+'/test/'+'axial/' + e + '.npy')
  os.rename(data_path + '/train/'+'coronal/' + e + '.npy', data_path+'/test/'+'coronal/' + e + '.npy')
  os.rename(data_path + '/train/'+'sagittal/' + e + '.npy', data_path+'/test/'+'sagittal/' + e + '.npy')